TTS(Text To Speech) : 텍스트 입력 => 음성 출력

In [ ]:
import gradio as gr
import requests
import os
from dotenv import load_dotenv
# .env 환경변수 로드
load_dotenv()


with gr.Blocks() as demo : 

    # 텍스트 => 음성으로 변환 요청 후, 응답받은 HEX로 임시 파일(mp3) 생성 후 경로 반환
    def request_tts(input_text) :
        # 1. Endpoint
        endpoint = "https://eastus.tts.speech.microsoft.com/cognitiveservices/v1"

        # 2. Header
        headers = {
            "Content-Type" : "application/ssml+xml",
            "Ocp-Apim-Subscription-Key" : os.getenv("STT_OCP_APIM_SUB_KEY"),
            "X-Microsoft-OutputFormat" : "audio-16khz-128kbitrate-mono-mp3"
        }

        # 3. Body : SSML
        body = f"""
        <speak version='1.0' xml:lang='ko-KR'>
            <voice name="ko-KR-SoonBokNeural" style="customerservice" rate="1.4">
                {input_text}
            </voice>
        </speak>
        """
    
        # request Audio => Text
        response = requests.post(endpoint, headers=headers, data=body)

        import tempfile
        with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmpfile:
            tmpfile.write(response.content) # 응답의 바이너리 콘텐츠(오디오 데이터)를 파일에 씁니다.
            temp_filepath = tmpfile.name
        
        return temp_filepath
    


    with gr.Row() :
        input_text = gr.Textbox(label="Input Text")                          # 텍스트 박스
        output_audio = gr.Audio(label="합성된 오디오 결과", type="filepath")    # 변환된 오디오 파일    


    # 텍스트 =>  음성 파일 변환 요청 버튼
    change_button = gr.Button("변환")
    change_button.click(fn=request_tts, inputs=[input_text], outputs=[output_audio])

demo.launch()

#request_tts("테스트 텍스트입니다.")
    

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
